In [66]:
!pip install --user pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 1.4 MB/s eta 0:00:0000:0100:01


In [90]:
import pandas as pd
operoniscore=pd.read_csv('scoresoperoni.txt',sep='\t',header=None,skiprows=1)
operoniscore.columns=["sequence_name","operone","lunghezza_intergenica","score","score_refined","motivo","posizione","info_geni"]
df_ann=pd.read_csv("/home/davide/Desktop/CCMEE29Annotazioni.csv")
df=pd.merge(operoniscore,df_ann,on="sequence_name")
#sort the dataframe by the column score_refined
df=df.sort_values(by="score_refined",ascending=False)
#rename sequence_name column
df=df.rename(columns={"sequence_name":"locus_tag_inizio"})
#save as excel
df.to_excel("scoresoperoni.xlsx",index=False)
df

,locus_tag_inizio,operone,lunghezza_intergenica,score,score_refined,motivo,posizione,info_geni,protein_id,product,gene,GO_process,GO_component,GO_function,GO_process_pannzer2,KEGG_brite,KEGG_pathways
1275,LAU37_RS16010,WP_250121493.1,300,10.52,21.04,TAGTATATCTGCACTA,-67,Gene WP_250121493.1. Score ortologhi= TS-821: ...,WP_250121493.1,recombinase RecA,recA,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","['GO:0009432 - SOS response', 'GO:0006310 - DN...",ko03400 DNA repair and recombination proteins,03440 Homologous recombination
1142,LAU37_RS14470,WP_250121220.1,128,10.22,20.22,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...,WP_250121220.1,transcriptional repressor LexA,lexA,['GO:0006282 - regulation of DNA repair [Evide...,NaN,['GO:0004252 - serine-type endopeptidase activ...,NaN,"ko01002 Peptidases and inhibitors, ko03400 DNA...",NaN
1154,LAU37_RS14595,WP_250121243.1,300,10.43,19.29,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...,WP_250121243.1,ERF family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3010,LAU37_RS21505,WP_250122517.1,300,9.18,17.92,GAGTACACTTGTACTG,-29,Gene WP_250122517.1. Score ortologhi= TS-821: ...,WP_250122517.1,single-stranded DNA-binding protein,NaN,NaN,NaN,NaN,GO:0006260 - DNA replication,"ko03032 DNA replication proteins, ko03400 DNA ...","03030 DNA replication, 03430 Mismatch repair, ..."
4876,LAU37_RS30680,WP_250126418.1,284,10.16,17.83,CAGTACATTTGTACTA,-137,Gene WP_250126418.1. Score ortologhi= TS-821: ...,WP_250126418.1,VOC family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,LAU37_RS11500,WP_250125700.1,31,-7.09,-107.09,TAGGCAATCGTCAGGA,-25,NaN,WP_250125700.1,nucleoside deaminase,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4573,LAU37_RS02825,WP_250124129.1,24,-7.29,-107.29,AAATGGAAGGTGTCAA,-20,NaN,WP_250124129.1,response regulator,NaN,['GO:0000160 - phosphorelay signal transductio...,NaN,NaN,NaN,NaN,NaN
4646,LAU37_RS02020,WP_250123972.1,24,-7.40,-107.40,ACGTCTCAATCAGGAA,-24,NaN,WP_250123972.1,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3507,LAU37_RS15185,WP_250121353.1,25,-8.02,-108.02,TTTTGAAACAGGAATC,-21,NaN,WP_250121353.1,glutathione S-transferase family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
#make a dictionary with locus_tag_inizio as key 
#extract GO terms from the GO_function column and form GO_process_pannzer2 column
import re
import pandas as pd
import polars as pl
import numpy as np

import numpy as np
#extract GO:XXXXXXX from the GO_function column
def extract_GO(row):
    if pd.isna(row):
        return np.nan
    else:
        return re.findall(r"GO:\d+",row)


def combine_lists(row, col1="GO_process_pannzer2",col2="GO_NCBI_process"):
    if isinstance(row[col1], list) and isinstance(row[col2], list):
        return row[col1] + row[col2]
    elif isinstance(row[col1], list):
        return row[col1]
    elif isinstance(row[col2], list):
        return row[col2]
    else:
        return np.nan

# Assuming df is your DataFrame and 'col1' and 'col2' are your columns

df["GO_process_pannzer2"]=df["GO_process_pannzer2"].apply(extract_GO)
df["GO_NCBI_process"]=df["GO_process"].apply(extract_GO)
#replace Nan with empty list
df["GO_process_pannzer2"]= [ [] if x is np.NaN else x for x in df["GO_process_pannzer2"] ]
df["GO_NCBI_process"]= [ [] if x is np.NaN else x for x in df["GO_NCBI_process"] ]
df
df["GO_process"]=df["GO_NCBI_process"]+df["GO_process_pannzer2"]
df["GO_process"]=df["GO_process"].apply(lambda x: list(set(x)))
df=df[["locus_tag_inizio","operone","GO_process"]]
df
#reindex the dataframe
df.reset_index(drop=True,inplace=True)
#save as pickle
import pickle
with open("GOsoperoni.pkl","wb") as f:
    pickle.dump(df,f)


In [89]:
#make  alist of GOs for first 50 rows
GOs=df.loc[0:50,"GO_process"].to_list()
GOs_background=df.loc[:,"GO_process"].to_list()
GOs_background=[item for sublist in GOs_background for item in sublist]
GOs=[item for sublist in GOs for item in sublist]
print(len(GOs))
print(len(GOs_background))
from collections import Counter
print(Counter(GOs))

34
2284
Counter({'GO:0006310': 3, 'GO:0006260': 3, 'GO:0006281': 2, 'GO:0006355': 2, 'GO:0009432': 1, 'GO:0006282': 1, 'GO:0032508': 1, 'GO:0071897': 1, 'GO:0006302': 1, 'GO:0015074': 1, 'GO:0019684': 1, 'GO:0022900': 1, 'GO:0006139': 1, 'GO:0006950': 1, 'GO:0006569': 1, 'GO:0051716': 1, 'GO:0006352': 1, 'GO:2000142': 1, 'GO:0006629': 1, 'GO:0000160': 1, 'GO:0016310': 1, 'GO:0008150': 1, 'GO:0015707': 1, 'GO:0042128': 1, 'GO:0015706': 1, 'GO:0001510': 1, 'GO:0006396': 1, 'GO:0055085': 1})


In [1]:
from pathlib import Path
dir=Path("/home/davide/Desktop/genomiChro/proteine_RefSeq/ortologhiCCMEE29")
#iterate over all files in the directory and make a read_csv with "\t" separator and make merge with the previous one on the first column
import pandas as pd
import numpy as np
import os

dfs=[]
for file in dir.iterdir():
    if file.is_file():
        df=pd.read_csv(file, sep="\t", header=None)
        #pick only the first two columns
        df=df.iloc[:,0:2]
        #rename columns
        df.columns=["ID", file.stem]
        #find duplicated in ID column and keep only the first one
        df=df[~df.duplicated(subset="ID", keep="first")]
        dfs.append(df)
#merge all the dataframes on ID
df_final=dfs[0]
for df in dfs[1:]:
    df_final=pd.merge(df_final, df, on="ID", how="outer")
df_final
#save the final dataframe


,ID,result_Chroococcidiopsis_sp._[FACHB-1243]_GCF_014696895.1.faa,result_Chroococcidiopsis_sp._CCNUC1_GCF_023546805.1.faa,result_Chroococcidiopsis_sp._CCMEE_29_GCF_023558375.1.faa,result_Chroococcidiopsis_thermalis_PCC_7203_GCF_000317125.1.faa,result_Chroococcidiopsis_cubana_SAG_39.79_GCF_034438355.1.faa,result_Chroococcidiopsis_sp._CCALA_051_GCF_003015105.1.faa,result_Chroococcidiopsis_sp._TS-821_GCF_002939305.1.faa,result_Chroococcidiopsis_cubana_CCALA_043_GCF_003003835.1.faa,result_Chroococcidiopsis_sp._SAG_2025_GCF_032860985.1.faa
0,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_009632395.1,WP_039717719.1,WP_039717719.1,WP_039717719.1,WP_104545246.1,WP_039717719.1,WP_039717719.1
1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_041548398.1,WP_006196724.1,WP_006196724.1,WP_006196724.1,WP_006276978.1,WP_006196724.1,WP_006196724.1
2,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_073591590.1,WP_015153091.1,WP_015153091.1,WP_015153091.1,WP_104546135.1,WP_015153091.1,WP_015153091.1
3,WP_190736661.1,WP_199317029.1,NaN,WP_190736661.1,NaN,WP_106168925.1,WP_106218980.1,NaN,WP_106168925.1,WP_317112279.1
4,WP_250121152.1,NaN,NaN,WP_250121152.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5780,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_275983394.1,WP_015154006.1,WP_015154006.1,WP_015154006.1,WP_104546721.1,WP_015154006.1,WP_015154006.1
5781,WP_275983395.1,NaN,NaN,WP_275983395.1,NaN,NaN,NaN,NaN,NaN,NaN
5782,WP_275983396.1,WP_192156346.1,WP_250017462.1,WP_275983396.1,WP_015163153.1,WP_199755664.1,WP_106545471.1,WP_104545432.1,WP_199755664.1,WP_039719372.1
5783,WP_275983397.1,NaN,NaN,WP_275983397.1,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#extract coding sequences from gbk file
from Bio import SeqIO
from pathlib import Path
from Bio.Seq import Seq
gbk_dir=Path("/home/davide/Desktop/genomiChro/annotati_Refseq")
def extract_cds(gbk):
    seqs={}
    
    for record in SeqIO.parse(gbk,"genbank"):
        for feature in record.features:
            if feature.type=="CDS" and "protein_id" in feature.qualifiers:
                pid=feature.qualifiers["protein_id"][0]
                col=[column for column in df_final.columns if gbk.stem in column][0]
                # Extract the series
                series = df_final.loc[df_final[col] == pid, "ID"]
                
                # Check if the series is empty
                if not series.empty:
                    pidCCMEE29 = series.values[0]
                    seqs[pidCCMEE29] = str(feature.extract(record.seq))
                else:
                    continue
    return seqs
cds={}

for gbk in gbk_dir.iterdir():
    if gbk.suffix==".gbff":
        cds[gbk.stem]=extract_cds(gbk)


In [4]:
print(len(cds))
for key, value in cds.items():
    print(len(key),len(value))
    print(key)


9
10 3354
FACHB-1243
9 3535
SAG_39.79
9 3364
CCALA_051
8 3385
PCC_7203
9 3414
CCALA_043
8 3520
SAG_2025
8 5785
CCMEE_29
6 3428
CCNUC1
6 3048
TS-821


In [3]:
#extract random coding sequences with len>intergenica and choose a random subsequence of len=intergenica
import random
seqs=[]
intergenica = 100  # replace with your value
cdsCCMEE29 = cds["CCMEE_29"]
def extract_300_random(lunghezza,cdsCCMEE29):
    seqs=[]
    for key in cdsCCMEE29:
        if len(cdsCCMEE29[key])>lunghezza:
            seqs.append(cdsCCMEE29[key])
    random_seqs=[]
    for i in range(300):
        seq=random.choice(seqs)
        start=random.randint(0,len(seq)-lunghezza)
        random_seqs.append(seq[start:start+lunghezza])
    return random_seqs
random_seqs=extract_300_random(intergenica,cdsCCMEE29)



In [10]:
with open("/home/davide/PycharmProjects/TESI2/cds.pkl", "rb") as f:
    cds = pickle.load(f)
len(cds)

4256

In [6]:
from collections import defaultdict
joined_dict = defaultdict(list)

# Iterate over each item in the outer dictionary and each item in the inner dictionaries
for outer_key, inner_dict in cds.items():
    if outer_key == "CCMEE_29":
        continue
    for key, value in inner_dict.items():
        # Append the value to the list of values for the key in the new dictionary
        joined_dict[key].append(value)

# Convert back to regular dict for final output
joined_dict = dict(joined_dict)



In [7]:
len(joined_dict)

4256

In [8]:
#save the dictionary as cds.pkl
import pickle
with open("cds.pkl","wb") as f:
    pickle.dump(joined_dict,f)

In [9]:
#save cdsCCMEE29 as cdsCCMEE29.pkl
with open("cdsCCMEE29.pkl","wb") as f:
    pickle.dump(cdsCCMEE29,f)

In [13]:
df=pd.read_csv("scoresoperoni.txt",sep="\t",header=None,skiprows=1)
df
#sort the dataframe by the 4th column
df=df.sort_values(by=4,ascending=False)
df

,0,1,2,3,4,5,6,7
1275,LAU37_RS16010,WP_250121493.1,300,10.52,21.04,TAGTATATCTGCACTA,-67,Gene WP_250121493.1. Score ortologhi= TS-821: ...
1142,LAU37_RS14470,WP_250121220.1,128,10.22,20.22,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...
1154,LAU37_RS14595,WP_250121243.1,300,10.43,19.29,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...
3010,LAU37_RS21505,WP_250122517.1,300,9.18,17.92,GAGTACACTTGTACTG,-29,Gene WP_250122517.1. Score ortologhi= TS-821: ...
4876,LAU37_RS30680,WP_250126418.1,284,10.16,17.83,CAGTACATTTGTACTA,-137,Gene WP_250126418.1. Score ortologhi= TS-821: ...
...,...,...,...,...,...,...,...,...
891,LAU37_RS11500,WP_250125700.1,31,-7.09,-107.09,TAGGCAATCGTCAGGA,-25,NaN
4573,LAU37_RS02825,WP_250124129.1,24,-7.29,-107.29,AAATGGAAGGTGTCAA,-20,NaN
4646,LAU37_RS02020,WP_250123972.1,24,-7.40,-107.40,ACGTCTCAATCAGGAA,-24,NaN
3507,LAU37_RS15185,WP_250121353.1,25,-8.02,-108.02,TTTTGAAACAGGAATC,-21,NaN


In [14]:
#drop the column after the 5th
df=df.drop(columns=[5,6,7])
#rename the columns
df.columns=["locus_tag","operone","len_intergenica","score","score_refined"]

In [16]:
df
#save as list of dictionaries
diz=df.to_dict("records")
diz

[{'locus_tag': 'LAU37_RS16010',
  'operone': 'WP_250121493.1',
  'len_intergenica': 300,
  'score': 10.52,
  'score_refined': 21.04},
 {'locus_tag': 'LAU37_RS14470',
  'operone': 'WP_250121220.1',
  'len_intergenica': 128,
  'score': 10.22,
  'score_refined': 20.22},
 {'locus_tag': 'LAU37_RS14595',
  'operone': 'WP_250121243.1',
  'len_intergenica': 300,
  'score': 10.43,
  'score_refined': 19.29},
 {'locus_tag': 'LAU37_RS21505',
  'operone': 'WP_250122517.1',
  'len_intergenica': 300,
  'score': 9.18,
  'score_refined': 17.92},
 {'locus_tag': 'LAU37_RS30680',
  'operone': 'WP_250126418.1',
  'len_intergenica': 284,
  'score': 10.16,
  'score_refined': 17.83},
 {'locus_tag': 'LAU37_RS16180',
  'operone': 'WP_250121527.1',
  'len_intergenica': 300,
  'score': 10.54,
  'score_refined': 16.69},
 {'locus_tag': 'LAU37_RS16880',
  'operone': 'WP_250121656.1|WP_250121655.1',
  'len_intergenica': 123,
  'score': 9.11,
  'score_refined': 15.71},
 {'locus_tag': 'LAU37_RS19450',
  'operone': 'WP_

In [126]:
import pandas as pd
operoniscore=pd.read_csv('scoresoperoni.txt',sep='\t',header=None,skiprows=1)
operoniscore.columns=["sequence_name","operone","lunghezza_intergenica","score","score_refined","motivo","posizione","info_geni"]
df_ann=pd.read_csv("/home/davide/Desktop/CCMEE29Annotazioni.csv")
df=pd.merge(operoniscore,df_ann,on="sequence_name")
#sort the dataframe by the column score_refined
df=df.sort_values(by="score_refined",ascending=False)
#rename sequence_name column
df=df.rename(columns={"sequence_name":"locus_tag_inizio"})
#save as excel
df.to_excel("scoresoperoni.xlsx",index=False)
df

,locus_tag_inizio,operone,lunghezza_intergenica,score,score_refined,motivo,posizione,info_geni,protein_id,product,gene,GO_process,GO_component,GO_function,GO_process_pannzer2,KEGG_brite,KEGG_pathways
1275,LAU37_RS16010,WP_250121493.1,300,10.52,21.04,TAGTATATCTGCACTA,-67,Gene WP_250121493.1. Score ortologhi= TS-821: ...,WP_250121493.1,recombinase RecA,recA,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","['GO:0009432 - SOS response', 'GO:0006310 - DN...",ko03400 DNA repair and recombination proteins,03440 Homologous recombination
1142,LAU37_RS14470,WP_250121220.1,128,10.22,20.22,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...,WP_250121220.1,transcriptional repressor LexA,lexA,['GO:0006282 - regulation of DNA repair [Evide...,NaN,['GO:0004252 - serine-type endopeptidase activ...,NaN,"ko01002 Peptidases and inhibitors, ko03400 DNA...",NaN
1154,LAU37_RS14595,WP_250121243.1,300,10.43,19.29,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...,WP_250121243.1,ERF family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3010,LAU37_RS21505,WP_250122517.1,300,9.18,17.92,GAGTACACTTGTACTG,-29,Gene WP_250122517.1. Score ortologhi= TS-821: ...,WP_250122517.1,single-stranded DNA-binding protein,NaN,NaN,NaN,NaN,GO:0006260 - DNA replication,"ko03032 DNA replication proteins, ko03400 DNA ...","03030 DNA replication, 03430 Mismatch repair, ..."
4876,LAU37_RS30680,WP_250126418.1,284,10.16,17.83,CAGTACATTTGTACTA,-137,Gene WP_250126418.1. Score ortologhi= TS-821: ...,WP_250126418.1,VOC family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,LAU37_RS11015,WP_250125605.1,23,-8.68,-8.86,AATGACTCCTAGGAGA,-21,Gene WP_250125605.1. Score ortologhi= TS-821: ...,WP_250125605.1,magnesium transporter,NaN,['GO:0006812 - monoatomic cation transport [Ev...,NaN,['GO:0008324 - monoatomic cation transmembrane...,NaN,ko02000 Transporters,NaN
340,LAU37_RS04125,WP_250124362.1,31,-8.18,-8.90,GCGAGATTCCCTACCG,-29,Gene WP_250124362.1. Score ortologhi= TS-821: ...,WP_250124362.1,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2285,LAU37_RS28140,WP_250126602.1,26,-8.99,-9.14,TGCAATCCAGGGAGCA,-21,Gene WP_250126602.1. Score ortologhi= SAG_39.7...,WP_250126602.1,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,LAU37_RS01475,WP_250123865.1,33,-5.87,-9.23,GAGACTATTAATATTG,-19,Gene WP_250123865.1. Score ortologhi= TS-821: ...,WP_250123865.1,ATP-binding protein,NaN,NaN,NaN,NaN,"['GO:0016310 - phosphorylation', 'GO:0000160 -...",NaN,NaN


In [102]:
#save as pickle
import pickle
with open("scoresoperoni.pkl","wb") as f:
    pickle.dump(diz,f)

In [103]:
import polars as pl
dfpvalue=pl.read_csv("provastats.txt",separator="\t",has_header=False)
#create numpy arrays from the columns
import numpy as np
array1=dfpvalue["column_1"].to_numpy()
scores=dfpvalue["column_2"].to_numpy()
scoresrefined=dfpvalue["column_3"].to_numpy()

In [127]:
def calculate_pvalue(score):
    n=len(scores)
    pvalues=[]
    pvalues.append(np.sum(scores>score)/n)
    x=np.sum(scores>score)/n
    return x
def calculate_pvalue_refined(score_refined):
    #calculate number of elements in the array with score>score_refined
    n=len(scores)
    pvalues=[]
    pvalues.append(np.sum(scoresrefined>score_refined)/n)
    x=np.sum(scoresrefined>score_refined)/n
    return x
#now apply the function to the dataframe to create two new columns pvalue and pvalue_refined
df["pvalue"]=df["score"].apply(calculate_pvalue)
df["pvalue_refined"]=df["score_refined"].apply(calculate_pvalue_refined)
df


    

,locus_tag_inizio,operone,lunghezza_intergenica,score,score_refined,motivo,posizione,info_geni,protein_id,product,gene,GO_process,GO_component,GO_function,GO_process_pannzer2,KEGG_brite,KEGG_pathways,pvalue,pvalue_refined
1275,LAU37_RS16010,WP_250121493.1,300,10.52,21.04,TAGTATATCTGCACTA,-67,Gene WP_250121493.1. Score ortologhi= TS-821: ...,WP_250121493.1,recombinase RecA,recA,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","['GO:0009432 - SOS response', 'GO:0006310 - DN...",ko03400 DNA repair and recombination proteins,03440 Homologous recombination,0.000000,0.000000
1142,LAU37_RS14470,WP_250121220.1,128,10.22,20.22,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...,WP_250121220.1,transcriptional repressor LexA,lexA,['GO:0006282 - regulation of DNA repair [Evide...,NaN,['GO:0004252 - serine-type endopeptidase activ...,NaN,"ko01002 Peptidases and inhibitors, ko03400 DNA...",NaN,0.000000,0.000000
1154,LAU37_RS14595,WP_250121243.1,300,10.43,19.29,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...,WP_250121243.1,ERF family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
3010,LAU37_RS21505,WP_250122517.1,300,9.18,17.92,GAGTACACTTGTACTG,-29,Gene WP_250122517.1. Score ortologhi= TS-821: ...,WP_250122517.1,single-stranded DNA-binding protein,NaN,NaN,NaN,NaN,GO:0006260 - DNA replication,"ko03032 DNA replication proteins, ko03400 DNA ...","03030 DNA replication, 03430 Mismatch repair, ...",0.000040,0.000000
4876,LAU37_RS30680,WP_250126418.1,284,10.16,17.83,CAGTACATTTGTACTA,-137,Gene WP_250126418.1. Score ortologhi= TS-821: ...,WP_250126418.1,VOC family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,LAU37_RS11015,WP_250125605.1,23,-8.68,-8.86,AATGACTCCTAGGAGA,-21,Gene WP_250125605.1. Score ortologhi= TS-821: ...,WP_250125605.1,magnesium transporter,NaN,['GO:0006812 - monoatomic cation transport [Ev...,NaN,['GO:0008324 - monoatomic cation transmembrane...,NaN,ko02000 Transporters,NaN,0.999554,0.999789
340,LAU37_RS04125,WP_250124362.1,31,-8.18,-8.90,GCGAGATTCCCTACCG,-29,Gene WP_250124362.1. Score ortologhi= TS-821: ...,WP_250124362.1,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999112,0.999800
2285,LAU37_RS28140,WP_250126602.1,26,-8.99,-9.14,TGCAATCCAGGGAGCA,-21,Gene WP_250126602.1. Score ortologhi= SAG_39.7...,WP_250126602.1,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999711,0.999866
125,LAU37_RS01475,WP_250123865.1,33,-5.87,-9.23,GAGACTATTAATATTG,-19,Gene WP_250123865.1. Score ortologhi= TS-821: ...,WP_250123865.1,ATP-binding protein,NaN,NaN,NaN,NaN,"['GO:0016310 - phosphorylation', 'GO:0000160 -...",NaN,NaN,0.990187,0.999886


In [128]:
df.to_excel("scoresoperoni.xlsx",index=False)

In [109]:
def calculate_pvalues(row):
    #calculate number of elements in the array with score>score_refined
    score=row["score"]
    return score*2
df["score2"]=df.apply(lambda x: calculate_pvalues)

In [110]:
df

,locus_tag_inizio,operone,lunghezza_intergenica,score,score_refined,motivo,posizione,info_geni,protein_id,product,gene,GO_process,GO_component,GO_function,GO_process_pannzer2,KEGG_brite,KEGG_pathways,score2
1275,LAU37_RS16010,WP_250121493.1,300,10.52,21.04,TAGTATATCTGCACTA,-67,Gene WP_250121493.1. Score ortologhi= TS-821: ...,WP_250121493.1,recombinase RecA,recA,"['GO:0006281 - DNA repair [Evidence IEA]', 'GO...",['GO:0005737 - cytoplasm [Evidence IEA]'],"['GO:0003677 - DNA binding [Evidence IEA]', 'G...","['GO:0009432 - SOS response', 'GO:0006310 - DN...",ko03400 DNA repair and recombination proteins,03440 Homologous recombination,NaN
1142,LAU37_RS14470,WP_250121220.1,128,10.22,20.22,TAGTACGAATGTTCTA,-48,Gene WP_250121220.1. Score ortologhi= TS-821: ...,WP_250121220.1,transcriptional repressor LexA,lexA,['GO:0006282 - regulation of DNA repair [Evide...,NaN,['GO:0004252 - serine-type endopeptidase activ...,NaN,"ko01002 Peptidases and inhibitors, ko03400 DNA...",NaN,NaN
1154,LAU37_RS14595,WP_250121243.1,300,10.43,19.29,TAGTACATATGTACTA,-208,Gene WP_250121243.1. Score ortologhi= SAG_39.7...,WP_250121243.1,ERF family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3010,LAU37_RS21505,WP_250122517.1,300,9.18,17.92,GAGTACACTTGTACTG,-29,Gene WP_250122517.1. Score ortologhi= TS-821: ...,WP_250122517.1,single-stranded DNA-binding protein,NaN,NaN,NaN,NaN,GO:0006260 - DNA replication,"ko03032 DNA replication proteins, ko03400 DNA ...","03030 DNA replication, 03430 Mismatch repair, ...",NaN
4876,LAU37_RS30680,WP_250126418.1,284,10.16,17.83,CAGTACATTTGTACTA,-137,Gene WP_250126418.1. Score ortologhi= TS-821: ...,WP_250126418.1,VOC family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,LAU37_RS11500,WP_250125700.1,31,-7.09,-107.09,TAGGCAATCGTCAGGA,-25,NaN,WP_250125700.1,nucleoside deaminase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4573,LAU37_RS02825,WP_250124129.1,24,-7.29,-107.29,AAATGGAAGGTGTCAA,-20,NaN,WP_250124129.1,response regulator,NaN,['GO:0000160 - phosphorelay signal transductio...,NaN,NaN,NaN,NaN,NaN,NaN
4646,LAU37_RS02020,WP_250123972.1,24,-7.40,-107.40,ACGTCTCAATCAGGAA,-24,NaN,WP_250123972.1,hypothetical protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3507,LAU37_RS15185,WP_250121353.1,25,-8.02,-108.02,TTTTGAAACAGGAATC,-21,NaN,WP_250121353.1,glutathione S-transferase family protein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
operoniscore

,sequence_name,operone,lunghezza_intergenica,score,score_refined,motivo,posizione,info_geni
0,LAU37_RS00015,WP_250123596.1,300,3.87,3.72,CAGTGTAATTGCTCGG,-184,Gene WP_250123596.1. Score ortologhi= SAG_39.7...
1,LAU37_RS00020,WP_250123597.1,49,-3.79,-3.91,CAGCAATCAATTTCTT,-43,Gene WP_250123597.1. Score ortologhi= SAG_39.7...
2,LAU37_RS00025,WP_250123598.1,79,-2.62,-2.35,TTGGACATAACTGCTT,-76,Gene WP_250123598.1. Score ortologhi= SAG_39.7...
3,LAU37_RS00030,WP_250123599.1,297,0.98,1.07,GAAAATACCTGAACTT,-34,Gene WP_250123599.1. Score ortologhi= SAG_39.7...
4,LAU37_RS00035,WP_250123600.1|WP_250123601.1,130,-2.91,-3.23,TTCTTCCCCTGCTCCC,-73,Gene WP_250123600.1. Score ortologhi= SAG_39.7...
...,...,...,...,...,...,...,...,...
5105,LAU37_RS27645,WP_250126821.1,300,3.35,-96.65,TCGTAAATCTGCACCT,-295,NaN
5106,LAU37_RS27630,WP_250126534.1,300,0.39,0.77,ATGCACTGCTGTACGC,-264,Gene WP_250126534.1. Score ortologhi= SAG_39.7...
5107,LAU37_RS27625,WP_250126533.1|WP_250126532.1,70,-3.82,-5.14,GTGTAAGTTGCCCCCG,-69,Gene WP_250126533.1. Score ortologhi= TS-821: ...
5108,LAU37_RS27615,WP_250126531.1,90,-0.89,-0.89,TAGCATTCACGCCCAG,-73,Gene WP_250126531.1. Score ortologhi= TS-821: ...
